From the first notebook, in the "EDA" section, we saw that the main dependencies 

- age of cohort (month since acquisition Date) => the main dependence is on this
                => even more, it tends to become stable (no significant change) after the first 6 months
- Product
- Country
- how recent it is ("date of acquisition")
- the month it has been acquired (specially, Product B)

1- Following this, we will construct a simple baseline method that calculates the rate of change of the retention rate at each month since acquisition for each Country x Product (in the code, we calculate a "diff_retention_rate" per Product x Country x month since acquisition).

=> we obtained a kind of template (or curve) which describes the variation of the retention rate at each time step for each Country x Product

2- We use previous retrieved template to forecast retention rates in the future 

3- From the forecasted retention rates, we retrieved the Net Customer by multiplying "forecasted retention rates" * "initial Cohort size"

### Imports

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from itertools import product
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
import warnings
warnings.filterwarnings("ignore")


### Inputs

Note: to assess the quality of the model, please change the variable "CV_mode" to True ("Cross validation mode") and run all the notebook, the result can be seen in the last 3 cells (R2 89%m MAE 4.52) 

In [96]:
CV_mode=False ##Cross validation mode: If true, we execute this notebook and we get the performance at the end

MIN_AGE_FOR_TRAINING = 6 #starting which month after acquisition we use to calculate the slope (rate) of change of retention rate

DATE_SPLIT_TRAIN_VALID = "2020-10-31" 
DATE_SPLIT_PAST_FUTURE = '2021-04-30'

# Calculate the difference in months
date_diff_months = (pd.to_datetime(DATE_SPLIT_PAST_FUTURE).year - pd.to_datetime(DATE_SPLIT_TRAIN_VALID).year) * 12\
    + (pd.to_datetime(DATE_SPLIT_PAST_FUTURE).month - pd.to_datetime(DATE_SPLIT_TRAIN_VALID).month)

date_diff_months

6

In [51]:
def get_date_int(df, column):
   year = df[column].dt.year
   month = df[column].dt.month
   return year, month

In [52]:
df = pd.read_csv('df_actuals.csv',parse_dates=[0,1]) #calculated in 1st notebook "1_forecast_case..."
print(f"Shape: {df.shape}\n\ndtypes:\n{df.dtypes}\n")
df.head()

Shape: (3654, 18)

dtypes:
Date                             datetime64[ns]
Intake Month                     datetime64[ns]
Country                                  object
Product                                  object
Net Customers                             int64
months_since_acquisition                  int64
churn_customers                         float64
Date_month                                int64
Date_year                                 int64
Cohort_month                              int64
Cohort_year                               int64
Cohort_size                               int64
Cohort_name                              object
retention_rate                          float64
diff_retention_rate                     float64
retention_rate_last                     float64
months_since_acquisition_last             int64
Type                                     object
dtype: object



,Date,Intake Month,Country,Product,Net Customers,months_since_acquisition,churn_customers,Date_month,Date_year,Cohort_month,Cohort_year,Cohort_size,Cohort_name,retention_rate,diff_retention_rate,retention_rate_last,months_since_acquisition_last,Type
0,2019-01-31,2019-01-31,US,A,110,0,0.0,1,2019,1,2019,110,Jan-19,100.000000,0.000000,51.818182,27,actual
1,2019-02-28,2019-01-31,US,A,88,1,22.0,2,2019,1,2019,110,Jan-19,80.000000,-20.000000,51.818182,27,actual
2,2019-03-31,2019-01-31,US,A,83,2,5.0,3,2019,1,2019,110,Jan-19,75.454545,-4.545455,51.818182,27,actual
3,2019-04-30,2019-01-31,US,A,77,3,6.0,4,2019,1,2019,110,Jan-19,70.000000,-5.454545,51.818182,27,actual
4,2019-05-31,2019-01-31,US,A,75,4,2.0,5,2019,1,2019,110,Jan-19,68.181818,-1.818182,51.818182,27,actual


In [53]:
if CV_mode:
    
    df_train = df.loc[df['Date']<=DATE_SPLIT_TRAIN_VALID].copy()
    df_valid = df.loc[df['Date']>DATE_SPLIT_TRAIN_VALID].copy()

else:
    
    df_train=df.copy()


print(f"Shape: {df_train.shape}")
df_train

Shape: (2277, 18)


,Date,Intake Month,Country,Product,Net Customers,months_since_acquisition,churn_customers,Date_month,Date_year,Cohort_month,Cohort_year,Cohort_size,Cohort_name,retention_rate,diff_retention_rate,retention_rate_last,months_since_acquisition_last,Type
0,2019-01-31,2019-01-31,US,A,110,0,0.0,1,2019,1,2019,110,Jan-19,100.000000,0.000000,51.818182,27,actual
1,2019-02-28,2019-01-31,US,A,88,1,22.0,2,2019,1,2019,110,Jan-19,80.000000,-20.000000,51.818182,27,actual
2,2019-03-31,2019-01-31,US,A,83,2,5.0,3,2019,1,2019,110,Jan-19,75.454545,-4.545455,51.818182,27,actual
3,2019-04-30,2019-01-31,US,A,77,3,6.0,4,2019,1,2019,110,Jan-19,70.000000,-5.454545,51.818182,27,actual
4,2019-05-31,2019-01-31,US,A,75,4,2.0,5,2019,1,2019,110,Jan-19,68.181818,-1.818182,51.818182,27,actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3610,2020-09-30,2020-08-31,GB,C,47,1,8.0,9,2020,8,2020,55,Aug-20,85.454545,-14.545455,67.272727,8,actual
3611,2020-10-31,2020-08-31,GB,C,44,2,3.0,10,2020,8,2020,55,Aug-20,80.000000,-5.454545,67.272727,8,actual
3618,2020-09-30,2020-09-30,GB,C,56,0,0.0,9,2020,9,2020,56,Sep-20,100.000000,0.000000,58.928571,7,actual
3619,2020-10-31,2020-09-30,GB,C,41,1,15.0,10,2020,9,2020,56,Sep-20,73.214286,-26.785714,58.928571,7,actual


#### Calculate slope of decay of retention rate in train

In [54]:
df_train['Intake Month'].max()

Timestamp('2020-10-31 00:00:00')

In [55]:
# MIN_AGE_FOR_TRAINING

In [56]:
##recalculate for train

df_train['retention_rate_last'] = df_train.groupby(['Product','Country','Intake Month'])['retention_rate'].transform('min')
df_train['months_since_acquisition_last'] = df_train.groupby(['Product','Country','Intake Month'])['months_since_acquisition'].transform('max')
df_train


,Date,Intake Month,Country,Product,Net Customers,months_since_acquisition,churn_customers,Date_month,Date_year,Cohort_month,Cohort_year,Cohort_size,Cohort_name,retention_rate,diff_retention_rate,retention_rate_last,months_since_acquisition_last,Type
0,2019-01-31,2019-01-31,US,A,110,0,0.0,1,2019,1,2019,110,Jan-19,100.000000,0.000000,55.454545,21,actual
1,2019-02-28,2019-01-31,US,A,88,1,22.0,2,2019,1,2019,110,Jan-19,80.000000,-20.000000,55.454545,21,actual
2,2019-03-31,2019-01-31,US,A,83,2,5.0,3,2019,1,2019,110,Jan-19,75.454545,-4.545455,55.454545,21,actual
3,2019-04-30,2019-01-31,US,A,77,3,6.0,4,2019,1,2019,110,Jan-19,70.000000,-5.454545,55.454545,21,actual
4,2019-05-31,2019-01-31,US,A,75,4,2.0,5,2019,1,2019,110,Jan-19,68.181818,-1.818182,55.454545,21,actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3610,2020-09-30,2020-08-31,GB,C,47,1,8.0,9,2020,8,2020,55,Aug-20,85.454545,-14.545455,80.000000,2,actual
3611,2020-10-31,2020-08-31,GB,C,44,2,3.0,10,2020,8,2020,55,Aug-20,80.000000,-5.454545,80.000000,2,actual
3618,2020-09-30,2020-09-30,GB,C,56,0,0.0,9,2020,9,2020,56,Sep-20,100.000000,0.000000,73.214286,1,actual
3619,2020-10-31,2020-09-30,GB,C,41,1,15.0,10,2020,9,2020,56,Sep-20,73.214286,-26.785714,73.214286,1,actual


In [57]:
# df_train.to_csv('df_trainn.csv',index=False)

In [58]:
##retention rate
df_train_parameters = df_train.copy()
# df_train_parameters['retention_rate_last'] = df_train_parameters.groupby(['Product','Country','Intake Month'])['retention_rate'].transform('min')
# df_train_parameters['months_since_acquisition_last'] = df_train_parameters.groupby(['Product','Country','Intake Month'])['months_since_acquisition'].transform('max')

mask = (df_train['months_since_acquisition']>=MIN_AGE_FOR_TRAINING)

df_train_parameters = df_train.loc[mask].copy()
df_train_parameters['retention_rate_min_age'] = df_train_parameters.groupby(['Product','Country','Intake Month'])['retention_rate'].transform('max')

df_train_parameters['lag_1_retention_rate'] = df.groupby(['Product','Country','Intake Month'])['retention_rate'].shift(1).fillna(100)

df_train_parameters = df_train_parameters[['Product'
                                           ,'Country'
                                           ,'Intake Month'
                                           ,'retention_rate_min_age'
                                           ,'retention_rate_last'
                                           ,'months_since_acquisition_last']]\
                                               .dropna().drop_duplicates()
                                               
df_train_parameters['delta_retention_last_to_6m'] = df_train_parameters['retention_rate_last'] - df_train_parameters['retention_rate_min_age'] 
df_train_parameters['months_since_6m'] = df_train_parameters['months_since_acquisition_last'] - MIN_AGE_FOR_TRAINING
df_train_parameters['retention_slope'] = df_train_parameters['delta_retention_last_to_6m'] / df_train_parameters['months_since_6m']
df_train_parameters = df_train_parameters.reset_index(drop=True)
df_train_parameters.head(4)

,Product,Country,Intake Month,retention_rate_min_age,retention_rate_last,months_since_acquisition_last,delta_retention_last_to_6m,months_since_6m,retention_slope
0,A,US,2019-01-31,64.545455,55.454545,21,-9.090909,15,-0.606061
1,A,US,2019-02-28,63.953488,55.813953,20,-8.139535,14,-0.581395
2,A,US,2019-03-31,60.869565,53.260870,19,-7.608696,13,-0.585284
3,A,US,2019-04-30,58.407080,52.212389,18,-6.194690,12,-0.516224


In [59]:
# df_train_parameters.to_csv('df_tr_param_test_2.csv',index=False)

### Get forecast of intake (either for validation or for the test -next 24 months-)

In [60]:
##Create forecast template
# cartesian_product_intake = list(product(forecast_dates,products, countries))
# df_forecast_intake = pd.DataFrame(cartesian_product_intake, columns=['Date', 'Product', 'Country'])
# df_forecast_intake['Intake'] = 0
# df_forecast_intake.to_csv('df_forecast_intake.csv',index=False)

if CV_mode:
    df_forecast_intake = df_valid[df_valid['months_since_acquisition']==0].copy() 
    df_forecast_intake['retention_rate_last'] = 100 #new Cohorts have always 100% rate when starting
    df_forecast_intake['months_since_acquisition_last'] = 0 #and 0 months as age

else:
# Import forecast
##Note: File 'saved_forecast_intake.csv' was generated directly in Excel by eye-balling the actuals in 2019 vs 2020
##      It could have been calculated automatically by fitting a Lin Reg model like: Y(month) = m*Y(month-12) + b
##      which would yield m=1 and b=delta between same month from 1 year to the other
    df_forecast_intake = pd.read_csv('saved_forecast_intake.csv'
                                     ,usecols=['Date','Country','Product','Intake','Type'],parse_dates=[0])
print(f"shape df forecast: {df_forecast_intake.shape}")
df_forecast_intake.head()

shape df forecast: (54, 18)


,Date,Intake Month,Country,Product,Net Customers,months_since_acquisition,churn_customers,Date_month,Date_year,Cohort_month,Cohort_year,Cohort_size,Cohort_name,retention_rate,diff_retention_rate,retention_rate_last,months_since_acquisition_last,Type
385,2020-11-30,2020-11-30,US,A,142,0,0.0,11,2020,11,2020,142,Nov-20,100.0,0.0,100,0,actual
391,2020-12-31,2020-12-31,US,A,173,0,0.0,12,2020,12,2020,173,Dec-20,100.0,0.0,100,0,actual
396,2021-01-31,2021-01-31,US,A,134,0,0.0,1,2021,1,2021,134,Jan-21,100.0,0.0,100,0,actual
400,2021-02-28,2021-02-28,US,A,110,0,0.0,2,2021,2,2021,110,Feb-21,100.0,0.0,100,0,actual
403,2021-03-31,2021-03-31,US,A,116,0,0.0,3,2021,3,2021,116,Mar-21,100.0,0.0,100,0,actual


In [61]:
# df_forecast_intake.to_csv('fi_valid.csv',index=False)

### Create template of DatexCountryxProductxCohort to forecast in the next 24 months

In [62]:
if CV_mode:
    start_period = pd.to_datetime(DATE_SPLIT_TRAIN_VALID) + pd.DateOffset(months=1)
    end_period = pd.to_datetime(start_period) + pd.DateOffset(months=date_diff_months)
    
else:
    start_period = pd.to_datetime(DATE_SPLIT_PAST_FUTURE) + pd.DateOffset(months=1)
    end_period = pd.to_datetime(start_period) + pd.DateOffset(months=23)

forecast_dates = pd.date_range(start=start_period, end=end_period, freq='M')

print(forecast_dates)

DatetimeIndex(['2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
               '2021-03-31', '2021-04-30'],
              dtype='datetime64[ns]', freq='M')


In [63]:
# intake_months_old_cohorts = df['Intake Month'].unique().tolist()
intake_months_old_cohorts = df_train['Intake Month'].unique().tolist()
if CV_mode:
    intake_months_new_cohorts = df_forecast_intake['Intake Month'].unique().tolist()
else:
    intake_months_new_cohorts = df_forecast_intake['Date'].unique().tolist()
    
intake_months = intake_months_old_cohorts + intake_months_new_cohorts

products = df['Product'].unique()
countries = df['Country'].unique()

# Create a Cartesian product of the lists
cartesian_product_net_customers = list(product(forecast_dates, intake_months, products, countries))

# Create a DataFrame from the Cartesian product
df_forecast_net_customers = pd.DataFrame(cartesian_product_net_customers, columns=['Date', 'Intake Month', 'Product', 'Country'])

df_forecast_net_customers = df_forecast_net_customers[df_forecast_net_customers['Intake Month']<=df_forecast_net_customers['Date']].reset_index(drop=True) #remove the lines where 'Intake Month' > 'Date'
df_forecast_net_customers

,Date,Intake Month,Product,Country
0,2020-11-30,2019-01-31,A,US
1,2020-11-30,2019-01-31,A,CA
2,2020-11-30,2019-01-31,A,GB
3,2020-11-30,2019-01-31,B,US
4,2020-11-30,2019-01-31,B,CA
...,...,...,...,...
1372,2021-04-30,2021-04-30,B,CA
1373,2021-04-30,2021-04-30,B,GB
1374,2021-04-30,2021-04-30,C,US
1375,2021-04-30,2021-04-30,C,CA


In [64]:
df_forecast_net_customers['Cohort_type'] = 'old'

if CV_mode:
    df_forecast_net_customers.loc[df_forecast_net_customers['Intake Month']>DATE_SPLIT_TRAIN_VALID,'Cohort_type'] = 'new'

else:
    df_forecast_net_customers.loc[df_forecast_net_customers['Intake Month']>DATE_SPLIT_PAST_FUTURE,'Cohort_type'] = 'new'

df_forecast_net_customers['Type'] = 'forecast'

df_forecast_net_customers

,Date,Intake Month,Product,Country,Cohort_type,Type
0,2020-11-30,2019-01-31,A,US,old,forecast
1,2020-11-30,2019-01-31,A,CA,old,forecast
2,2020-11-30,2019-01-31,A,GB,old,forecast
3,2020-11-30,2019-01-31,B,US,old,forecast
4,2020-11-30,2019-01-31,B,CA,old,forecast
...,...,...,...,...,...,...
1372,2021-04-30,2021-04-30,B,CA,new,forecast
1373,2021-04-30,2021-04-30,B,GB,new,forecast
1374,2021-04-30,2021-04-30,C,US,new,forecast
1375,2021-04-30,2021-04-30,C,CA,new,forecast


In [65]:
# df_forecast_net_customers[df_forecast_net_customers['Cohort_size'].isnull()]

In [66]:
date_year, date_month = get_date_int(df_forecast_net_customers, 'Date')

cohort_year, cohort_month = get_date_int(df_forecast_net_customers, 'Intake Month')

# Calculate difference in years
years_diff = date_year - cohort_year

# Calculate difference in months
months_diff = date_month - cohort_month

# Extract the difference in months from all previous values

df_forecast_net_customers['months_since_acquisition'] = years_diff * 12 + months_diff
df_forecast_net_customers.head()

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition
0,2020-11-30,2019-01-31,A,US,old,forecast,22
1,2020-11-30,2019-01-31,A,CA,old,forecast,22
2,2020-11-30,2019-01-31,A,GB,old,forecast,22
3,2020-11-30,2019-01-31,B,US,old,forecast,22
4,2020-11-30,2019-01-31,B,CA,old,forecast,22


In [67]:
if CV_mode:

    df_forecast_train = df_forecast_net_customers.loc[df_forecast_net_customers['Intake Month']<=DATE_SPLIT_TRAIN_VALID]
    df_forecast_valid = df_forecast_net_customers.loc[df_forecast_net_customers['Intake Month']>DATE_SPLIT_TRAIN_VALID]

else:
    df_forecast_train = df_forecast_net_customers[df_forecast_net_customers['Cohort_type']=='old']
    df_forecast_test = df_forecast_net_customers[df_forecast_net_customers['Cohort_type']=='new']
    
df_forecast_train

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition
0,2020-11-30,2019-01-31,A,US,old,forecast,22
1,2020-11-30,2019-01-31,A,CA,old,forecast,22
2,2020-11-30,2019-01-31,A,GB,old,forecast,22
3,2020-11-30,2019-01-31,B,US,old,forecast,22
4,2020-11-30,2019-01-31,B,CA,old,forecast,22
...,...,...,...,...,...,...,...
1318,2021-04-30,2020-10-31,B,CA,old,forecast,6
1319,2021-04-30,2020-10-31,B,GB,old,forecast,6
1320,2021-04-30,2020-10-31,C,US,old,forecast,6
1321,2021-04-30,2020-10-31,C,CA,old,forecast,6


##### Get Cohort size for the Forecast parts

In [68]:
df_forecast_train = df_forecast_train.merge(df[['Product'
                                                ,'Country'
                                                ,'Intake Month'
                                                ,'Cohort_size']].drop_duplicates()
                                            ,how='left',on=['Product','Country','Intake Month'])

if CV_mode:
    df_forecast_valid = df_forecast_valid.merge(
                    df[['Product','Country','Intake Month','Cohort_size']]
                    .drop_duplicates()
                    ,how='left'
                    ,on=['Product','Country','Intake Month'])
else:
    df_forecast_test = df_forecast_test.merge(df_forecast_intake,
                        how='left',
                        left_on=['Product','Country','Intake Month'],
                        right_on=['Product','Country','Date'])\
                        .drop(['Date_y','Type_y'],axis=1)\
                        .rename(columns={'Date_x':'Date','Intake':'Cohort_size','Type_x':'Type'})
                        
df_forecast_train.head()

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition,Cohort_size
0,2020-11-30,2019-01-31,A,US,old,forecast,22,110
1,2020-11-30,2019-01-31,A,CA,old,forecast,22,17
2,2020-11-30,2019-01-31,A,GB,old,forecast,22,96
3,2020-11-30,2019-01-31,B,US,old,forecast,22,0
4,2020-11-30,2019-01-31,B,CA,old,forecast,22,0


In [69]:
df_forecast_train = df_forecast_train.sort_values(by=['Product','Country','Intake Month','Date']).reset_index(drop=True)

if CV_mode:
    df_forecast_valid= df_forecast_valid.sort_values(by=['Product','Country','Intake Month','Date']).reset_index(drop=True)

df_forecast_train

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition,Cohort_size
0,2020-11-30,2019-01-31,A,CA,old,forecast,22,17
1,2020-12-31,2019-01-31,A,CA,old,forecast,23,17
2,2021-01-31,2019-01-31,A,CA,old,forecast,24,17
3,2021-02-28,2019-01-31,A,CA,old,forecast,25,17
4,2021-03-31,2019-01-31,A,CA,old,forecast,26,17
...,...,...,...,...,...,...,...,...
1183,2020-12-31,2020-10-31,C,US,old,forecast,2,61
1184,2021-01-31,2020-10-31,C,US,old,forecast,3,61
1185,2021-02-28,2020-10-31,C,US,old,forecast,4,61
1186,2021-03-31,2020-10-31,C,US,old,forecast,5,61


### Forecasting part: forecast retention rates of train dataset in the next 24 months


In this part we forecast the the future retention rates by using the slope after the first 6 months of acquisition

It will be used to calculate the diff of retention rate by Product x Country x Cohort x months since acquisition (age)

In [70]:
avg_ret_slopes = df_train_parameters.groupby(['Product','Country'])['retention_slope'].mean().reset_index()
avg_ret_slopes

,Product,Country,retention_slope
0,A,CA,-0.752444
1,A,GB,-0.605785
2,A,US,-0.635100
3,B,CA,-0.581662
4,B,GB,-0.640330
5,B,US,-0.579621
6,C,CA,-0.714977
7,C,GB,-0.602159
8,C,US,-0.559077


In [71]:
df_forecast_train = df_forecast_train.merge(df_train_parameters[['Product'
                                                                 ,'Country'
                                                                 ,'Intake Month'
                                                                 ,'months_since_acquisition_last'
                                                                 ,'retention_rate_last'
                                                                 ,'retention_slope']]
                                            ,how='left'
                                            ,on=['Product','Country','Intake Month'])

# df_forecast_train.to_csv('df_fcst_train_before.csv',index=False)

In [72]:
df_forecast_train['retention_slope'] = df_forecast_train['retention_slope'].fillna(
    df_forecast_train.merge(avg_ret_slopes,on=['Product', 'Country'],how='left')['retention_slope_y']
    
)

In [73]:
for col in ['months_since_acquisition_last','retention_rate_last']:
    
    df_forecast_train[col] = df_forecast_train[col].fillna(
        df_forecast_train.merge(df_train[['Product'
                                          , 'Country'
                                          ,'Intake Month'
                                          ,'months_since_acquisition_last'
                                          ,'retention_rate_last']].drop_duplicates(), on=['Product', 'Country','Intake Month'], how='left')[f'{col}_y']
    )

In [74]:
# df_forecast_train.loc[df_forecast_train['retention_slope']==0] = df_forecast_train.loc[df_forecast_train['retention_slope']==0].replace(0,
#     df_forecast_train.merge(avg_ret_slopes,on=['Product', 'Country'],how='left')['retention_slope_y']
    
# )

df_forecast_train['retention_slope'] = np.where(df_forecast_train['retention_slope'] == 0, 
                                df_forecast_train.merge(avg_ret_slopes, on=['Product', 'Country'], how='left')['retention_slope_y'], 
                                df_forecast_train['retention_slope'])

In [75]:
# df_forecast_train.to_csv('df_fcst_train_after_v2.csv',index=False)

In [76]:
# for col in ['months_since_acquisition_last','retention_rate_last']:
    
#     df_forecast_train[col] = df_forecast_train[col].fillna(
#         df_forecast_train.merge(df_train[['Product', 'Country','Intake Month','months_since_acquisition_last','retention_rate_last']].drop_duplicates(), on=['Product', 'Country','Intake Month'], how='left')[f'{col}_y']
#     )

In [77]:
# df_forecast_train.to_csv('df_fcst_train_after_v3.csv',index=False)

In [78]:
df_forecast_train['delta_months_for_forecast'] = df_forecast_train['months_since_acquisition'] - df_forecast_train['months_since_acquisition_last']
df_forecast_train['forecast_retention_rates'] = df_forecast_train['retention_rate_last'] + df_forecast_train['delta_months_for_forecast'] * df_forecast_train['retention_slope']

df_forecast_train.head()

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition,Cohort_size,months_since_acquisition_last,retention_rate_last,retention_slope,delta_months_for_forecast,forecast_retention_rates
0,2020-11-30,2019-01-31,A,CA,old,forecast,22,17,21.0,52.941176,-0.784314,1.0,52.156863
1,2020-12-31,2019-01-31,A,CA,old,forecast,23,17,21.0,52.941176,-0.784314,2.0,51.372549
2,2021-01-31,2019-01-31,A,CA,old,forecast,24,17,21.0,52.941176,-0.784314,3.0,50.588235
3,2021-02-28,2019-01-31,A,CA,old,forecast,25,17,21.0,52.941176,-0.784314,4.0,49.803922
4,2021-03-31,2019-01-31,A,CA,old,forecast,26,17,21.0,52.941176,-0.784314,5.0,49.019608


In [79]:
# df_forecast_train.to_csv('df_fcst_train_after_v4.csv',index=False)

In [80]:
df_forecast_train['forecast_net_customers'] = df_forecast_train['forecast_retention_rates']/100 * df_forecast_train['Cohort_size']
df_forecast_train['forecast_net_customers'] = df_forecast_train['forecast_net_customers'].fillna(0).round()


df_forecast_train.head()

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition,Cohort_size,months_since_acquisition_last,retention_rate_last,retention_slope,delta_months_for_forecast,forecast_retention_rates,forecast_net_customers
0,2020-11-30,2019-01-31,A,CA,old,forecast,22,17,21.0,52.941176,-0.784314,1.0,52.156863,9.0
1,2020-12-31,2019-01-31,A,CA,old,forecast,23,17,21.0,52.941176,-0.784314,2.0,51.372549,9.0
2,2021-01-31,2019-01-31,A,CA,old,forecast,24,17,21.0,52.941176,-0.784314,3.0,50.588235,9.0
3,2021-02-28,2019-01-31,A,CA,old,forecast,25,17,21.0,52.941176,-0.784314,4.0,49.803922,8.0
4,2021-03-31,2019-01-31,A,CA,old,forecast,26,17,21.0,52.941176,-0.784314,5.0,49.019608,8.0


In [81]:
# df_forecast_train.to_csv('fcst_train.csv',index=False)

In [82]:
# df.loc[(df['Product']=='A')&(df['Country']=='CA')&(df['Intake Month']=='2019-01-31')].to_csv('act.csv',index=False)

### Get retention rates for train actuals and forecasted

In [83]:
df_train_retention_rates_all = pd.concat([df_train[['Product'
                                                    ,'Country'
                                                    ,'Date'
                                                    ,'Intake Month'
                                                    ,'months_since_acquisition'
                                                    ,'retention_rate','Type']]
                                          ,df_forecast_train[['Product'
                                                              ,'Country'
                                                              ,'Date'
                                                              ,'Intake Month'
                                                              ,'months_since_acquisition'
                                                              ,'forecast_retention_rates'
                                                              ,'Type']].rename(columns={'forecast_retention_rates':'retention_rate'})]
                                         ,axis=0)
df_train_retention_rates_all

,Product,Country,Date,Intake Month,months_since_acquisition,retention_rate,Type
0,A,US,2019-01-31,2019-01-31,0,100.000000,actual
1,A,US,2019-02-28,2019-01-31,1,80.000000,actual
2,A,US,2019-03-31,2019-01-31,2,75.454545,actual
3,A,US,2019-04-30,2019-01-31,3,70.000000,actual
4,A,US,2019-05-31,2019-01-31,4,68.181818,actual
...,...,...,...,...,...,...,...
1183,C,US,2020-12-31,2020-10-31,2,98.881846,forecast
1184,C,US,2021-01-31,2020-10-31,3,98.322770,forecast
1185,C,US,2021-02-28,2020-10-31,4,97.763693,forecast
1186,C,US,2021-03-31,2020-10-31,5,97.204616,forecast


In [84]:
df_train_retention_rates_all = df_train_retention_rates_all.sort_values(by=['Product','Country','Intake Month','Date'],ascending=True).reset_index(drop=True)

df_train_retention_rates_all['diff_retention_rate'] = df_train_retention_rates_all.groupby(['Product','Country','Intake Month'])['retention_rate'].diff()

#we fillna with 0 but only on the ones where there is a retention rate (ex: for B Product, there is no retention rate so we keep NaN)
df_train_retention_rates_all.loc[~df_train_retention_rates_all['retention_rate'].isnull(),'diff_retention_rate'] = df_train_retention_rates_all.loc[~df_train_retention_rates_all['retention_rate'].isnull(),'diff_retention_rate'].fillna(0)

### Get mean retention rates per Country x Product x months since acquisition

In [85]:
fitted_diff_retention_rates = df_train_retention_rates_all.groupby(['Product','Country','months_since_acquisition'])['diff_retention_rate'].mean().ffill().reset_index() #the ffill is for Product B who hasn't have more than 40 something months since acquisition
fitted_diff_retention_rates = fitted_diff_retention_rates.rename(columns={'diff_retention_rate':'fitted_diff_retention_rate'})
fitted_diff_retention_rates.head()

,Product,Country,months_since_acquisition,fitted_diff_retention_rate
0,A,CA,0,0.000000
1,A,CA,1,-21.424954
2,A,CA,2,-4.070097
3,A,CA,3,-4.293303
4,A,CA,4,-1.576673


As we can see below, it respects the diff of retention rate of Product B for ex, which has always 100% rate in the 1st 3 months since acquisition

In [86]:
#check
fitted_diff_retention_rates[fitted_diff_retention_rates['Product']=='B']

,Product,Country,months_since_acquisition,fitted_diff_retention_rate
84,B,CA,0,0.000000
85,B,CA,1,0.000000
86,B,CA,2,0.000000
87,B,CA,3,-31.961489
88,B,CA,4,-1.018183
...,...,...,...,...
163,B,US,23,-0.668449
164,B,US,24,-0.668449
165,B,US,25,-0.668449
166,B,US,26,-0.668449


### If we are in CV mode, forecast valid as well

In [87]:
if CV_mode:
    df_valid_parameters = df_valid.copy()
    df_valid_parameters = df_valid_parameters[['Product','Country','Intake Month','retention_rate_last','months_since_acquisition_last']].dropna().drop_duplicates()
    df_valid_parameters.head(5)

In [88]:
if CV_mode:
    df_forecast_valid = df_forecast_valid.merge(df_valid_parameters,how='left',on=['Product','Country','Intake Month'])
    df_forecast_valid = df_forecast_valid.merge(fitted_diff_retention_rates,how='left',on=['Product','Country','months_since_acquisition'])

    df_forecast_valid['cum_fitted_retention_rate'] = df_forecast_valid.groupby(['Product','Country','Intake Month'])['fitted_diff_retention_rate'].cumsum()
    df_forecast_valid['forecast_retention_rates'] = df_forecast_valid['retention_rate_last'] + df_forecast_valid['cum_fitted_retention_rate'] 
    df_forecast_valid['forecast_net_customers'] = df_forecast_valid['forecast_retention_rates']/100 * df_forecast_valid['Cohort_size']
    df_forecast_valid['forecast_net_customers'] = df_forecast_valid['forecast_net_customers'].fillna(0).round()

else:
    df_forecast_test['retention_rate_last'] = 100 #a new Cohort starts always with 100% rate
    df_forecast_test['months_since_acquisition_last'] = 0 #and 0 as age

    df_forecast_test = df_forecast_test.merge(fitted_diff_retention_rates,how='left',on=['Product','Country','months_since_acquisition'])

    df_forecast_test['cum_fitted_retention_rate'] = df_forecast_test.groupby(['Product','Country','Intake Month'])['fitted_diff_retention_rate'].cumsum()
    df_forecast_test['forecast_retention_rates'] = df_forecast_test['retention_rate_last'] + df_forecast_test['cum_fitted_retention_rate'] 
    df_forecast_test['forecast_net_customers'] = df_forecast_test['forecast_retention_rates']/100 * df_forecast_test['Cohort_size']
    df_forecast_test['forecast_net_customers'] = df_forecast_test['forecast_net_customers'].fillna(0).round()

    df_forecast_test

In [89]:
cols = ['Date'
        ,'Intake Month'
        ,'Product'
        ,'Country'
        ,'Cohort_type'
        ,'Type'
        ,'months_since_acquisition'
        ,'Cohort_size'
        ,'forecast_retention_rates'
        ,'forecast_net_customers']


In [90]:
if CV_mode:
    df_forecast = pd.concat([df_forecast_train[cols],df_forecast_valid[cols]],axis=0)
else:
    df_forecast = pd.concat([df_forecast_train[cols],df_forecast_test[cols]],axis=0)


df_forecast = df_forecast.merge(df[['Product'
                                    ,'Country'
                                    ,'Intake Month'
                                    ,'Date'
                                    ,'Net Customers']]
                                ,how='left'
                                ,on=['Product','Country','Intake Month','Date'])
df_forecast

,Date,Intake Month,Product,Country,Cohort_type,Type,months_since_acquisition,Cohort_size,forecast_retention_rates,forecast_net_customers,Net Customers
0,2020-11-30,2019-01-31,A,CA,old,forecast,22,17,52.156863,9.0,9
1,2020-12-31,2019-01-31,A,CA,old,forecast,23,17,51.372549,9.0,9
2,2021-01-31,2019-01-31,A,CA,old,forecast,24,17,50.588235,9.0,9
3,2021-02-28,2019-01-31,A,CA,old,forecast,25,17,49.803922,8.0,9
4,2021-03-31,2019-01-31,A,CA,old,forecast,26,17,49.019608,8.0,9
...,...,...,...,...,...,...,...,...,...,...,...
1372,2021-03-31,2021-02-28,C,US,new,forecast,1,63,45.147344,28.0,45
1373,2021-04-30,2021-02-28,C,US,new,forecast,2,63,40.643675,26.0,42
1374,2021-03-31,2021-03-31,C,US,new,forecast,0,63,77.777778,49.0,63
1375,2021-04-30,2021-03-31,C,US,new,forecast,1,63,56.258455,35.0,49


In [91]:
df_forecast.to_csv('df_forecast_baseline.csv',index=False)

In [92]:
if CV_mode:
    y_true = df_forecast['Net Customers'].values
    y_pred = df_forecast['forecast_net_customers'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"MAE: {np.round(mae,3)}, R2: {np.round(r2,3)},, RMSE: {np.round(rmse,3)}")

MAE: 4.511, R2: 0.892,, RMSE: 14.24


In [93]:
# df_submission.to_csv('forecast_net_customers.csv',index=False)

In [94]:
cols_actuals = ['Product','Country','Intake Month','Date','retention_rate','Type','Net Customers']
cols_forecast = ['Product','Country','Intake Month','Date','forecast_retention_rates','Type','forecast_net_customers']

if CV_mode:
        
    df_all = pd.concat([df_train[cols_actuals]
            ,df_valid[cols_actuals]
            ,df_forecast_train[cols_forecast]
            ,df_forecast_valid[cols_forecast]
                        ]
                         ,axis=0)
else:
        df_all = pd.concat([df_train[cols_actuals]
            ,df_forecast_train[cols_forecast]
            ,df_forecast_test[cols_forecast]
                            ]
                             ,axis=0)

df_all['retention_rate'] = np.where(df_all['retention_rate'].isnull(),df_all['forecast_retention_rates'],df_all['retention_rate'])
df_all['Net Customers'] = np.where(df_all['Net Customers'].isnull(),df_all['forecast_net_customers'],df_all['Net Customers'])

df_all = df_all.drop(['forecast_retention_rates','forecast_net_customers'],axis=1)
print(f"shape: {df_all.shape}")
df_all

shape: (5031, 7)


,Product,Country,Intake Month,Date,retention_rate,Type,Net Customers
0,A,US,2019-01-31,2019-01-31,100.000000,actual,110.0
1,A,US,2019-01-31,2019-02-28,80.000000,actual,88.0
2,A,US,2019-01-31,2019-03-31,75.454545,actual,83.0
3,A,US,2019-01-31,2019-04-30,70.000000,actual,77.0
4,A,US,2019-01-31,2019-05-31,68.181818,actual,75.0
...,...,...,...,...,...,...,...
184,C,US,2021-02-28,2021-03-31,45.147344,forecast,28.0
185,C,US,2021-02-28,2021-04-30,40.643675,forecast,26.0
186,C,US,2021-03-31,2021-03-31,77.777778,forecast,49.0
187,C,US,2021-03-31,2021-04-30,56.258455,forecast,35.0


In [95]:
df_all.to_csv('df_all_net_customers_baseline.csv',index=False)